In [25]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [7]:
dataset = load_dataset("andresnowak/Instruction-finetuning-mnlp")

In [8]:
dataset["train"]["messages"][0][0]["content"]

'Provide a detailed analysis of Candace Parker\'s defensive techniques in her recent games, excluding the words "aggressive" and "blocking", in the format of a sports commentary script.'

In [19]:
def tokenize_function(example):
    val = example["messages"][0][0]["content"]

In [20]:
dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/264919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 264919
    })
})

In [26]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")

In [29]:
def format_instruction(example):
    """Extract the first user instruction and assistant response"""
    messages = example
    instruction = ""
    response = ""
    
    # Find first user message (instruction)
    for msg in messages:
        if msg['role'] == 'user':
            instruction = msg['content']
            break
    
    # Find corresponding assistant response
    for msg in messages:
        if msg['role'] == 'assistant':
            response = msg['content']
            break
    
    return {
        "input": instruction,
        "output": response,
        "formatted_text": f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    }

def tokenize_function(examples):
    # Format the examples
    formatted = [format_instruction(ex) for ex in examples["messages"]]
    
    # Tokenize inputs (instruction only)
    model_inputs = tokenizer(
        [f["input"] for f in formatted],
        truncation=True,
        max_length=1000,
        padding="max_length"
    )
    
    # Tokenize outputs (response only) for labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [f["output"] for f in formatted],
            truncation=True,
            max_length=1000,
            padding="max_length"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/264919 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/mcqa_model/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 264919
    })
})

: 